# Instance based Knowledge Graph Creation

In [1]:
import text_to_graph as ttg
import numpy as np

In [2]:
text = 'Darth Vader, also known by his birth name Anakin Skywalker, is a fictional character in the Star Wars franchise. Darth Vader appears in the original film trilogy as a pivotal antagonist whose actions drive the plot, while his past as Anakin Skywalker and the story of his corruption are central to the narrative of the prequel trilogy. The character was created by George Lucas and has been portrayed by numerous actors. His appearances span the first six Star Wars films, as well as Rogue One, and his character is heavily referenced in Star Wars: The Force Awakens. He is also an important character in the Star Wars expanded universe of television series, video games, novels, literature and comic books. Originally a Jedi who was prophesied to bring balance to the Force, he falls to the dark side of the Force and serves the evil Galactic Empire at the right hand of his Sith master, Emperor Palpatine (also known as Darth Sidious).'
text

'Darth Vader, also known by his birth name Anakin Skywalker, is a fictional character in the Star Wars franchise. Darth Vader appears in the original film trilogy as a pivotal antagonist whose actions drive the plot, while his past as Anakin Skywalker and the story of his corruption are central to the narrative of the prequel trilogy. The character was created by George Lucas and has been portrayed by numerous actors. His appearances span the first six Star Wars films, as well as Rogue One, and his character is heavily referenced in Star Wars: The Force Awakens. He is also an important character in the Star Wars expanded universe of television series, video games, novels, literature and comic books. Originally a Jedi who was prophesied to bring balance to the Force, he falls to the dark side of the Force and serves the evil Galactic Empire at the right hand of his Sith master, Emperor Palpatine (also known as Darth Sidious).'

### Named Entity Recognition

In [3]:
ner_dict = ttg.process_NER(text=text)
ner_dict

{'Darth Vader': 'PERSON',
 'Anakin Skywalker': 'PERSON',
 'George Lucas': 'PERSON',
 'first': 'ORDINAL',
 'six': 'CARDINAL',
 'Star Wars': 'WORK_OF_ART',
 'Star Wars: The Force Awakens': 'WORK_OF_ART',
 'Force': 'ORG',
 'Galactic Empire': 'ORG',
 'Sith': 'WORK_OF_ART',
 'Palpatine': 'PERSON',
 'Darth Sidious': 'PERSON'}

### Generate Coreferences and Dependencies

In [4]:
corefs = ttg.process_corefs(text=text, corenlp_path='./stanford-corenlp-4.2.0')
print("Coreferences found: ", len(corefs))

Coreferences found:  3


### Replacement with Named Entities and Dependencies

In [5]:
resolved_text = ttg.process_dependency_matching(text=text, ner_dict=ner_dict, corefs=corefs)
resolved_text

'Anakin Skywalker , is a fictional character in the Star Wars franchise. Anakin Skywalker appears in the original film trilogy as a pivotal antagonist whose actions drive the plot, while his past as Anakin Skywalker and the story of his corruption are central to the narrative of the prequel trilogy. Anakin Skywalker was created by George Lucas and has been portrayed by numerous actors. Anakin Skywalker appearances span the first six Star Wars films, as well as Rogue One, and his character is heavily referenced in Star Wars: The Force Awakens. Anakin Skywalker is also an important character in the Star Wars expanded universe of television series, video games, novels, literature and comic books. Originally a Jedi who was prophesied to bring balance to the Force, Anakin Skywalker falls to the dark side of the Force and serves the evil Galactic Empire at the right hand of his Sith master, Emperor Palpatine( also known as Darth Sidious) . '

### Add Neural Coref

In [6]:
resolved_text = ttg.process_neural_coref(resolved_text)
resolved_text

'Anakin Skywalker , is a fictional character in the Star Wars franchise. Anakin Skywalker appears in the original film trilogy as a pivotal antagonist whose actions drive the plot, while Anakin Skywalker past as Anakin Skywalker and the story of Anakin Skywalker corruption are central to the narrative of the original film trilogy. Anakin Skywalker was created by George Lucas and has been portrayed by numerous actors. Anakin Skywalker appearances span the first six Star Wars films, as well as Rogue One, and Anakin Skywalker character is heavily referenced in Star Wars: The Force Awakens. Anakin Skywalker is also an important character in the Star Wars expanded universe of television series, video games, novels, literature and comic books. Originally a Jedi who was prophesied to bring balance to the Force, Anakin Skywalker falls to the dark side of the Force and serves the evil Galactic Empire at the right hand of Anakin Skywalker Sith master, Emperor Palpatine( also known as Darth Sidio

### Relation Extraction with OpenIE

In [8]:
triples = ttg.process_relation_extraction(text=resolved_text)
triples

Starting server with command: java -Xmx8G -cp /Users/rhythmsyed/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7e08b452e898462b.props -preload openie


,subject,relation,object
0,Anakin Skywalker,is fictional character in,Star Wars franchise
1,Anakin Skywalker,is,fictional character
2,fictional character,is in,Star Wars franchise
3,Anakin Skywalker,appears as,pivotal antagonist
4,Anakin Skywalker,appears in,film trilogy
5,Anakin Skywalker past,are,central
6,Anakin Skywalker past,are central to,narrative
7,Anakin Skywalker past,are central to,narrative of original film trilogy
8,Anakin Skywalker,appears as,antagonist
9,actions,drive plot,central to narrative of film trilogy


### Pruning & Entity Linking

In [23]:
triple_df = ttg.process_triple_pruning(triples=triples, ner_dict=ner_dict)
triple_df = ttg.process_entity_linking(triple_df=triple_df, confidence_threshold=0.75)

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 1

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 2

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 3

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 4

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 5

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 6

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 7

Sentence 1: Star Wars franchise
Sentence 2: Star Wars
Similarity: tensor(0.8333)
Processed 8

Sentence 1: film trilogy
Sentence 2: original film trilogy
Similarity: tensor(0.7743)
Processed 9

Sentence 1: narrative of original film trilogy
Sentence 2: narrative of film trilogy
Similarity: tensor(0.9187)
Processed 10

Sentence 1: central to 

### Check Final Output Graph

In [24]:
triple_df

,Type1,Entity1,Relationship,Type2,Entity2
0,Node,Anakin Skywalker,is fictional character in,Node,Star Wars
1,Node,Anakin Skywalker,is,Node,fictional character
2,Node,fictional character,is in,Node,Star Wars
3,Node,Anakin Skywalker,appears as,Node,pivotal antagonist
4,Node,Anakin Skywalker,appears in,Node,film trilogy
5,Node,Anakin Skywalker,are,Node,central
6,Node,Anakin Skywalker,are central to,Node,narrative
7,Node,Anakin Skywalker,are central to,Node,narrative of film trilogy
8,Node,Anakin Skywalker,appears as,Node,antagonist
9,Node,actions,drive plot,Node,central to narrative of film trilogy


## Topic and Concept Extraction 

### Taxogen

In [7]:
ner_dict = ttg.process_NER(text=resolved_text)
entities = []
for entity in ner_dict.keys():
    entities.append(entity.replace(' ', '_'))
entities

['Anakin_Skywalker',
 'George_Lucas',
 'first',
 'six',
 'Star_Wars',
 'Star_Wars:_The_Force_Awakens',
 'Force',
 'Galactic_Empire',
 'Anakin_Skywalker_Sith',
 'Palpatine',
 'Darth_Sidious']

In [ ]:
np.savetxt('./taxogen/data/starwars/raw/keywords.txt', entities, fmt='%s')
np.savetxt('./taxogen/data/starwars/raw/papers.txt', entities, fmt='%s')

### Generate Word Embeddings with Word2Vec

https://wikipedia2vec.github.io/wikipedia2vec/pretrained/

In [11]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../word-embeddings/enwiki_20180420_300d.txt', binary=False)

In [19]:
# sentence = ['George_Lucas',
#  'first',
#  'six',
#  'Star_Wars',
#  'Star_Wars:_The_Force_Awakens',
#  'Force',
#  'Galactic_Empire',
#  'Anakin_Skywalker_Sith',
#  'Palpatine',
#  'Darth_Sidious']
# vectors = [model[w] for w in sentence]
# vectors

In [ ]:
! ./taxogen/code/word2vec -train ./taxogen/data/starwars/raw/keywords.txt -output ./taxogen/data/starwars/input/embeddings.txt

In [ ]:
! python taxogen/code/cluster-preprocess.py ./taxogen/data/starwars

In [ ]:
! python taxogen/code/preprocess.py ./taxogen/data/starwars

In [ ]:
! cp ./taxogen/data/starwars/input/embeddings.txt ./taxogen/data/starwars/init/embeddings.txt

In [ ]:
! cp ./taxogen/data/starwars/input/keywords.txt ./taxogen/data/starwars/init/seed_keywords.txt

In [ ]:
! python ./taxogen/code/main.py ./taxogen/data/starwars/